In [0]:
from pyspark.sql import Row

birth_rdd = sc.textFile("./FileStore/tables/birth.txt")
birth_split = birth_rdd.map(lambda x : x.split("/"))
birth_row = birth_split.map(lambda x : Row(name = x[0],year = x[1],month = x[2],day = x[3], city = x[4]))
birth_df = sqlContext.createDataFrame(birth_row)
birth_df.show()

+----+----+-----+---+-------+
name|year|month|day| city|
+----+----+-----+---+-------+
 tom|1996| 7| 21| seoul|
 sam|1994| 1| 20| busan|
 bob|2000| 12| 11| seoul|
 kim|2002| 6| 8|daejeon|
 lee|1998| 2| 10| busan|
+----+----+-----+---+-------+

In [0]:
peopleDF = spark.read.parquet("/FileStore/tables/people-10m.parquet")
peopleDF.show(5)

+---+---------+----------+----------+------+-------------------+-----------+------+
 id|firstName|middleName| lastName|gender| birthDate| ssn|salary|
+---+---------+----------+----------+------+-------------------+-----------+------+
 1| Pennie| Carry|Hirschmann| F|1955-07-02 04:00:00|981-43-9345| 56172|
 2| An| Amira| Cowper| F|1992-02-08 05:00:00|978-97-8086| 40203|
 3| Quyen| Marlen| Dome| F|1970-10-11 04:00:00|957-57-8246| 53417|
 4| Coralie| Antonina| Marshal| F|1990-04-11 04:00:00|963-39-4885| 94727|
 5| Terrie| Wava| Bonar| F|1980-01-16 05:00:00|964-49-8051| 79908|
+---+---------+----------+----------+------+-------------------+-----------+------+
only showing top 5 rows

In [0]:
import pyspark.sql.functions as f

peopleDF.filter("salary <= '20000'")\
        .withColumn("salary",f.abs("salary"))\
        .withColumn("new_salary",f.round(f.col("salary") * 1.1)).show(5)

+---+---------+----------+--------+------+-------------------+-----------+------+----------+
 id|firstName|middleName|lastName|gender| birthDate| ssn|salary|new_salary|
+---+---------+----------+--------+------+-------------------+-----------+------+----------+
317| Shirley| Deidre| Pikhno| F|1979-11-14 05:00:00|919-31-6376| 11393| 12532.0|
354| Samantha| Rema| Worms| F|1970-02-22 05:00:00|943-84-9242| 19249| 21174.0|
381| Eugenie| Shonna|Doonican| F|1989-01-25 05:00:00|954-16-8896| 6523| 7175.0|
468| Tarsha| Lina| Halse| F|1992-09-11 04:00:00|956-34-7851| 18105| 19916.0|
973| Kirstie| Breanne| Bulfoot| F|1979-05-20 04:00:00|970-44-3658| 8451| 9296.0|
+---+---------+----------+--------+------+-------------------+-----------+------+----------+
only showing top 5 rows

In [0]:
peopleDF.filter(f.col("salary") >= 30000)\
        .select(f.max("salary"),f.min("salary"),f.round(f.avg("salary")).alias("averageSalary")).show()

+-----------+-----------+-------------+
max(salary)|min(salary)|averageSalary|
+-----------+-----------+-------------+
 180841| 30000| 73471.0|
+-----------+-----------+-------------+

In [0]:
import pyspark.sql.functions as f

peopleDF.groupBy("lastName","gender")\
        .agg(f.count("salary").alias("count"),f.max("salary").alias("max salary"))\
        .show(5)

+-----------+------+-----+----------+
 lastName|gender|count|max salary|
+-----------+------+-----+----------+
 Aliman| F| 460| 130786|
Fareweather| F| 440| 113598|
 Vick| F| 380| 132846|
 Aulton| F| 384| 112703|
 Godmer| F| 384| 120555|
+-----------+------+-----+----------+
only showing top 5 rows

In [0]:
bikeSharingDayDF = (spark
                     .read
                     .option("inferSchema","true")
                     .option("header","true")
                     .csv("/FileStore/tables/day.csv"))

bikeSharingDayDF.show(3)

+-------+----------+------+---+----+-------+-------+----------+----------+--------+--------+--------+---------+------+----------+----+
instant| dteday|season| yr|mnth|holiday|weekday|workingday|weathersit| temp| atemp| hum|windspeed|casual|registered| cnt|
+-------+----------+------+---+----+-------+-------+----------+----------+--------+--------+--------+---------+------+----------+----+
 1|2011-01-01| 1| 0| 1| 0| 6| 0| 2|0.344167|0.363625|0.805833| 0.160446| 331| 654| 985|
 2|2011-01-02| 1| 0| 1| 0| 0| 0| 2|0.363478|0.353739|0.696087| 0.248539| 131| 670| 801|
 3|2011-01-03| 1| 0| 1| 0| 1| 1| 1|0.196364|0.189405|0.437273| 0.248309| 120| 1229|1349|
+-------+----------+------+---+----+-------+-------+----------+----------+--------+--------+--------+---------+------+----------+----+
only showing top 3 rows

In [0]:
databricksBlogDF = spark\
                  .read.option("inferSchema","true")\
                  .option("header","true")\
                  .json("/FileStore/tables/databricks_blog-2.json")

databricksBlogDF.show(5)

+--------------------+--------------------+--------------------+-------+--------------------+-----------+---+--------------------+--------------------+-------+--------------------+
 authors| categories| content|creator| dates|description| id| link| slug| status| title|
+--------------------+--------------------+--------------------+-------+--------------------+-----------+---+--------------------+--------------------+-------+--------------------+
[Tomer Shiran (VP...|[Company Blog, Pa...|<div class="post-...| roy|[2014-04-10, 2014...| null| 33|https://databrick...|mapr-integrates-s...|publish|MapR Integrates t...|
 [Tathagata Das]|[Apache Spark, En...|We are happy to a...| tdas|[2014-04-10, 2014...| null| 35|https://databrick...|spark-0_9_1-released|publish|Apache Spark 0.9....|
 [Steven Hillion]|[Company Blog, Pa...|<div class="post-...| roy|[2014-04-01, 2014...| null| 37|https://databrick...|application-spotl...|publish|Application Spotl...|
[Michael Armbrust...|[Apache Spark, En...|Building a unifie...|michael|[2014-03-27, 2014...| null| 42|https://databrick...|spark-sql-manipul...|publish|Spark SQL: Manipu...|
 [Patrick Wendell]|[Apache Spark, En...|Our goal with Apa...|patrick|[2014-02-04, 2014...| null| 58|https://databrick...| release-0_9_0|publish|Apache Spark 0.9....|
+--------------------+--------------------+--------------------+-------+--------------------+-----------+---+--------------------+--------------------+-------+--------------------+
only showing top 5 rows

In [0]:
BlogDF2 = databricksBlogDF.select('authors','categories','creator','dates')\
                          .withColumn('publishedOn',f.col("dates.publishedOn"))
BlogDF2.show(3)

+--------------------+--------------------+-------+--------------------+-----------+
 authors| categories|creator| dates|publishedOn|
+--------------------+--------------------+-------+--------------------+-----------+
[Tomer Shiran (VP...|[Company Blog, Pa...| roy|[2014-04-10, 2014...| 2014-04-10|
 [Tathagata Das]|[Apache Spark, En...| tdas|[2014-04-10, 2014...| 2014-04-10|
 [Steven Hillion]|[Company Blog, Pa...| roy|[2014-04-01, 2014...| 2014-04-01|
+--------------------+--------------------+-------+--------------------+-----------+
only showing top 3 rows

In [0]:
resultDF = databricksBlogDF.select('categories','creator',f.date_format("dates.publishedOn","yyyy-MM-dd").alias("date"))\
                            .filter(f.year(f.col("date")) == '2013')\
                            .orderBy(f.col("date"))
resultDF.show(3)

+--------------------+-------+----------+
 categories|creator| date|
+--------------------+-------+----------+
[Announcements, C...| ion|2013-10-27|
[Announcements, C...| matei|2013-10-28|
[Company Blog, Pa...| ion|2013-10-29|
+--------------------+-------+----------+
only showing top 3 rows

In [0]:
dataBricksBlog2DF = databricksBlogDF.\
                    select("title","authors",f.explode(f.col("authors")).alias("author")).\
                    filter(f.size(f.col("authors")) > 1).\
                    orderBy("title")
dataBricksBlog2DF.show(5)

+--------------------+--------------------+---------------+
 title| authors| author|
+--------------------+--------------------+---------------+
"Learning Spark" ...|[Holden Karau, An...|Patrick Wendell|
"Learning Spark" ...|[Holden Karau, An...| Matei Zaharia|
"Learning Spark" ...|[Holden Karau, An...| Andy Konwinski|
"Learning Spark" ...|[Holden Karau, An...| Holden Karau|
AMPLab updates th...|[Ahir Reddy, Reyn...| Ahir Reddy|
+--------------------+--------------------+---------------+
only showing top 5 rows